In [1]:
import pandas, sqlalchemy
from datetime import datetime

In [2]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

In [21]:
Base = automap_base()

In [22]:
engine = create_engine('postgres+psycopg2://postgres@localhost:5436/anherf')

In [23]:
Base.prepare(engine, reflect=True)

In [24]:
Base.classes.keys()

['alembic_version', 'registrations', 'participants', 'roles']

In [25]:
session = Session(engine)

In [20]:
speaker_role = Roles(desc='speaker')
guest_role = Roles(desc='guest')
vip_role = Roles(desc='vip')
participant_role = Roles(desc='participant')

In [21]:
session.add(speaker_role)
session.add(guest_role)
session.add(vip_role)
session.add(participant_role)
session.commit()

In [26]:
Roles = Base.classes['roles']

In [27]:
Participant = Base.classes['participants']

In [28]:
Registration = Base.classes['registrations']

In [29]:
def add_new_participant(role, data):
    for item in data.iterrows():
        row = item[1]
        existing_par = session.query(Participant).filter(Participant.id==row['ID']).first()
        if existing_par:
            continue
        new_par = Participant(old_id=row['ID'],
                        title=row['name_title'],
                         firstname=row['first_name'],
                         lastname=row['last_name'],
                         email=row['user_email'],
                         faculty=row['faculty'],
                         affil=row['university'],
                         mobile=row['mobile'],
                         delivery_address=row['delivery_address'],
                         position_type=row['position_type'],
                         role_id=role.id)
        registered_at = datetime.strptime(row['user_registered'], '%Y-%m-%d %H:%M:%S')
        new_par.registrations_collection.append(Registration(registered_at=registered_at,
                                                    pay_status=False, payment_required=False))
        session.add(new_par)
    session.commit()

In [30]:
committee_data = pandas.read_excel('register 04.10.2018.xlsx', sheet_name='committee')

In [31]:
speaker_data = pandas.read_excel('register 04.10.2018.xlsx', sheet_name='speaker')

In [37]:
participant_data = pandas.read_excel('register 04.10.2018.xlsx', sheet_name='participant')

In [33]:
speaker_role = session.query(Roles).filter(Roles.desc=='speaker').first()
add_new_participant(speaker_role, speaker_data)

In [35]:
committee_role = session.query(Roles).filter(Roles.desc=='committee').first()
add_new_participant(committee_role, committee_data)

In [38]:
participant_role = session.query(Roles).filter(Roles.desc=='participant').first()
add_new_participant(participant_role, participant_data)